Group Memebrs:


*   Trusha Sanghvi 202001188
*   Chaitri Gudkha 202003022
*   Nikhil Vaghasiya 202003042


In [ ]:
!pip install qiskit -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 10.6 MB/s eta 0:00:00


In [ ]:
!pip install pylatexenc -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import numpy as np
import math
from time import time
from math import gcd
from qiskit import(QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer)
from qiskit.circuit.library.standard_gates.u1 import MCU1Gate, U1Gate
import matplotlib
from qiskit.visualization import circuit_drawer
#from qiskit.aqua.circuits import FourierTransformCircuits as ftc
#from qiskit.aqua.circuits.gates import mcu1
from qiskit.quantum_info.operators import Operator
from qiskit.circuit.library import QFT
import pylatexenc
from qiskit.circuit.library.standard_gates import CPhaseGate

#from qiskit.aqua.utils import summarize_circuits

In [ ]:
def mod_exp(a, b, N):
    X = int(a)
    E = int(b)
    m = int(N)
    if(E<0 or m<1):
        return None
    X = X % m
    Y = 1
    while E > 0:
        if E % 2 == 0:
            X = (X * X) % m
            E = E/2
        else:
            Y = (X * Y) % m
            E = E - 1
    return Y

In [ ]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

In [ ]:
def modinv(a, n):
    if(n<2):
        return None
    g, _, x = egcd(n, a)
    if(g!=1):
        return None
    return x % n

In [ ]:
def order(a, N):
    N = int(N)
    a = int(a) % N
    if(N<2 or gcd(a, N)!=1):
        return None
    r = 1
    p = a
    while(p != 1):
        p = (p*a) % N
        r = r + 1
    return r

In [ ]:
def get_angles(a, n):
    bit = bin(abs(int(a)))[2:].zfill(n)
    theta = np.zeros(n)
    for i in range(n):
        for j in range(n-1-i, n):
            if bit[j] == '1':
                theta[i] += 2**(n-i-j-1)
    theta = theta * np.pi
    return theta

In [ ]:
def phi_add(circuit, q, a, n, c, inverse = False):
    theta = get_angles(a, n)
    for i in range(n):
        circuit.append(MCU1Gate(theta[i], len(c)), c[:] + [q[i]])
        # circuit.append(CPhaseGate(theta[i],ctrl_state=None), c[:1] + [q[i]])


            

In [ ]:
def phi_add_mod_N(circuit, q, a, n, N, c, aux, inverse = False):
    if N <= 0 or N >= 2**n :
        print('Choose appropriate N')
        return None
    a = a % N
    phi_add(circuit, q, N, n+1, c, [aux])

In [ ]:
def multiply_mod_N(circuit, q, a, n, N, c, aux):
    if N <= 0 or N >= 2**n :
        print('Choose appropriate N')
        return None
    a = a % N
    a_inv = modinv(a, N)
    if(not a_inv):
        print('Inverse of a does not exist')
        return None

    pow_2_mod = np.zeros(n)
    pow_2_mod[0] = 1
    for i in range(1, n):
        pow_2_mod[i] = (2*pow_2_mod[i-1]) % N

    for i in range(n):
        phi_add_mod_N(circuit, aux, (pow_2_mod[i]*a) % N, n, N, [c, q[i]], aux[n+1], aux[n+1])
    
    for i in range(n):
        circuit.cswap(c, q[i], aux[i])

In [ ]:
def power_mod_N(circuit, q, r, a, m, n, N, aux):
    if N <= 0 or N >= 2**n :
        print('Choose appropriate N')
        return None
    temp = a % N
    for i in range(m):
        multiply_mod_N(circuit, r, temp, n, N, q[i], aux)
        temp = (temp*temp) % N

In [ ]:
def discrete_log(a, b, N, noise_model=None, n_shots=32):
    N = int(N)
    a = int(a) % N
    b = int(b) % N
    n_shots = int(n_shots)
    
    if(N<2 or gcd(a, N) != 1 or n_shots<1):
        print('Input error')
        return None, None
    if(gcd(b, N) != 1):
        print('Input error')
        return None, None
    
    r = int(order(a, N))
    if(r==1):
        if(b==1):
            return 0, None
        else:
            print('Input error')
            return None, None
    m = int(np.ceil(np.log2(N+1)))
    #m=5 n=4
    n = int(np.ceil(np.log2(r+1))) + 1
    q = QuantumRegister(2*(m+n+1), 'q')
    c = ClassicalRegister(2*n, 'c')
    circuit = QuantumCircuit(q, c)
    
    for i in range(2*n):
        circuit.h(q[i])
    circuit.x(q[2*n])
    power_mod_N(circuit, q[:n], q[2*n:2*n+m], a, n, m, N, q[2*n+m:2*n+2*m+2])
    power_mod_N(circuit, q[n:2*n], q[2*n:2*n+m], b, n, m, N, q[2*n+m:2*n+2*m+2])
    
    circuit.append(QFT(n, do_swaps=False, inverse=True),range(n))
    circuit.append(QFT(n, do_swaps=False, inverse=True),range(n,2*n))
    
    circuit.measure(q[:2*n], c[:2*n])
    
    simulator = Aer.get_backend('qasm_simulator')
    job = execute(circuit, simulator, noise_model=noise_model, shots=n_shots)
    result = job.result()
    counts = result.get_counts(circuit)
    print(counts)
    M = len(counts)
     
    k = []
    kt_mod_r = []
    for key in counts:
        x = int(key[-n:], 2) 
        y = int(key[-2*n:-n], 2) 
        k.append(round((x*r)/(2**n)))
        kt_mod_r.append(round((y*r)/(2**n)))

    for j in range(M):
        for i in range(j+1):
            if(gcd(k[i], k[j]) == 1):
                _, v1, v2 = egcd(k[i], k[j])
                t = (kt_mod_r[i]*v1 + kt_mod_r[j]*v2)%r
                if(mod_exp(a, t, N) == b):
                    return t, circuit
    return None, circuit

In [ ]:
a = 6
b = 30
N = 31
n_shots = 32
#noise_model = get_noise(0.01, 0.01)
noise_model = None
s_time = time()
t,circuit = discrete_log(a, b, N, noise_model, n_shots)
print('t =', t)
print('Time:', round(time() - s_time), 's') 
# circuit.draw(output='mpl',initial_state=True, cregbundle=False)
# print(circuit)

{'01110011': 1, '11011001': 1, '10011011': 1, '10110011': 2, '10111011': 2, '11010101': 2, '01010011': 1, '00111101': 1, '01011011': 2, '00000000': 19}
t = 3
Time: 1 s


In [ ]:
circuit.draw(output='mpl',initial_state=True, cregbundle=False)